In [ ]:
import numpy as np
import math
import pandas as pd

# Part 1. Reading data

In [ ]:
train_filename = 'training_set.tsv'
test_filename = 'benchmark_set.tsv'

In [ ]:
train_df = pd.read_csv(train_filename, sep='\t')
test_df = pd.read_csv(test_filename, sep='\t')
train_df.head()

,UniProtKB accession,Taxa,Kingdom,Class,Cross-validation fold,Sequence (first 50 N-terminal residues),SP cleavage-site annotation
0,P61916,Homo sapiens (Human),Metazoa,SP,0,MRFLAATFLLLALSTAAQAEPVQFKDCGSVDGVIKEVNVSPCPTQP...,SSSSSSSSSSSSSSSSSSSNNNNNNNNNNNNNNNNNNNNNNNNNNN...
1,Q7M3V1,Chelonus sp. nr. curvimaculatus (Parasitic wasp),Metazoa,SP,0,MAGKEVIFIMALFIAVESSPIFSFDDLVCPSVTSLRVNVEKNECST...,SSSSSSSSSSSSSSSSSSSSNNNNNNNNNNNNNNNNNNNNNNNNNN...
2,Q08738,Bombyx mori (Silk moth),Metazoa,SP,0,MRVFLAICLSLTVALAAETGKYTPFQYNRVYSTVSPFVYKPGRYVA...,SSSSSSSSSSSSSSSSNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
3,Q41350,Solanum lycopersicum (Tomato) (Lycopersicon es...,Plants,SP,0,MASSSAKILLPLSLLFTLLSLSQSTNPNFILTLVNNCPYTIWPAIQ...,SSSSSSSSSSSSSSSSSSSSSSSSNNNNNNNNNNNNNNNNNNNNNN...
4,Q86SE1,Androctonus amoreuxi (African fattail scorpion...,Metazoa,SP,0,MNYLVMISLALLLMIGVESVRDGYIVYPHNCVYHCIPSCDGLCKEN...,SSSSSSSSSSSSSSSSSSSNNNNNNNNNNNNNNNNNNNNNNNNNNN...


In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1723 entries, 0 to 1722
Data columns (total 7 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   UniProtKB accession                      1723 non-null   object
 1   Taxa                                     1723 non-null   object
 2   Kingdom                                  1723 non-null   object
 3   Class                                    1723 non-null   object
 4   Cross-validation fold                    1723 non-null   int64 
 5   Sequence (first 50 N-terminal residues)  1723 non-null   object
 6   SP cleavage-site annotation              1723 non-null   object
dtypes: int64(1), object(6)
memory usage: 94.4+ KB


In [ ]:
train_df['Cross-validation fold'].unique()

array([0, 1, 2, 3, 4])

In [ ]:
train_df['Class'].unique()

array(['SP', 'NO_SP'], dtype=object)

# Part 2. VonHeijne method

In [ ]:
a = 'a' * 16
len(a[1:15])

14

In [ ]:
def build_pspm(df):
  aminos = ['S', 'P', 'V', 'T', 'C', 'Y', 'A', 'D', 'K', 'E', 'W', 'I', 'Q', 'R', 'G', 'M', 'N', 'F', 'L', 'H']
  pspm = [{amino:1 for amino in aminos} for x in range(15)]
  for i, row in df.iterrows():
    index = row['SP cleavage-site annotation'].find('N')
    clevage_site = row['Sequence (first 50 N-terminal residues)'][index-13:index+2]
    for j in range(15):
      pspm[j][clevage_site[j]] += 1
  return pspm

In [ ]:
def build_pswm(pspm, n):
  aminos = ['S', 'P', 'V', 'T', 'C', 'Y', 'A', 'D', 'K', 'E', 'W', 'I', 'Q', 'R', 'G', 'M', 'N', 'F', 'L', 'H']
  aminos_freq_swiss = {'S': 0.0664, 'P': 0.0474, 'V': 0.0686, 'T': 0.0535, 'C':0.0138, 'Y':0.0292, 'A':0.0825, 'D':0.0546,
                       'K': 0.0580, 'E': 0.0672, 'W': 0.0110, 'I': 0.0591, 'Q': 0.0393, 'R': 0.0553, 'G': 0.0707, 'M':0.0241,
                       'N': 0.0406, 'F': 0.0386, 'L': 0.0965, 'H': 0.0227}
  pswm = [{amino:1 for amino in aminos} for x in range(15)]
  for i in range(len(pspm)):
    for amino in aminos:
      pswm[i][amino] = np.log2((pspm[i][amino] / n) / aminos_freq_swiss[amino])
  return pswm

In [ ]:
def predict_score(s, pswm):
  max_w = -999999
  for i in range(15, len(s)):
    w = 0
    for j in range(i-15, i):
      w += pswm[j-i+15][s[j]]
    max_w = max(max_w, w)
  return max_w


In [ ]:
from sklearn.metrics import precision_recall_curve, f1_score, accuracy_score, matthews_corrcoef, classification_report


In [ ]:
def calculate_all_metrics(y_true, y_pred):
  report = classification_report(y_true, y_pred)
  print(report)
  precision, recall, tresholds = precision_recall_curve(y_true, y_pred)
  f_score = f1_score(y_true, y_pred)
  accuracy = accuracy_score(y_true, y_pred, normalize = True)
  mcc = matthews_corrcoef(y_true, y_pred)
  print(' MCC: {mcc}, Accuracy: {accuracy}, F1: {f_score}'.
        format(f_score=f_score,
              #  precision=precision,
              #  recall=recall,
               accuracy=accuracy,
               mcc=mcc))

In [ ]:
optimal_tresholds = []
for validation_fold_n in range(5):
  # training p.1
  trainig_fold_df = train_df.loc[train_df['Cross-validation fold'] != validation_fold_n]
  trainig_fold_true_values_df = trainig_fold_df.loc[trainig_fold_df['Class'] == 'SP']
  pspm = build_pspm(trainig_fold_true_values_df)
  n = len(trainig_fold_true_values_df) + 20
  pswm = build_pswm(pspm, n=n)
  # training p.2 get treshold
  y_true = trainig_fold_df['Class'].apply(lambda x: 1 if x == 'SP' else 0).values
  y_pred = trainig_fold_df['Sequence (first 50 N-terminal residues)'].apply(predict_score, pswm=pswm).values
  precision, recall, tresholds = precision_recall_curve(y_true, y_pred)
  f_score = (2 * precision * recall) / (precision + recall)
  index = np.argmax(f_score)
  optimal_treshold = tresholds[index]
  optimal_tresholds.append(optimal_treshold)
  # prediction
  validation_fold_df = train_df.loc[train_df['Cross-validation fold'] == validation_fold_n]
  y_test_true = validation_fold_df['Class'].apply(lambda x: 1 if x == 'SP' else 0).values
  y_test_pred = validation_fold_df['Sequence (first 50 N-terminal residues)'].apply(predict_score, pswm=pswm).values
  y_test_pred = [int(y >= optimal_treshold) for y in y_test_pred]
  # prediction p.2
  print('Optimal treshold for CV {} is {}'.format(validation_fold_n, optimal_treshold))
  calculate_all_metrics(y_test_true, y_test_pred)
  print('---------------------------------------------------------------------')

Optimal treshold for CV 0 is 7.928636845904684
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       293
           1       0.89      0.79      0.84        52

    accuracy                           0.95       345
   macro avg       0.93      0.89      0.90       345
weighted avg       0.95      0.95      0.95       345

 MCC: 0.8118929011894348, Accuracy: 0.9536231884057971, F1: 0.836734693877551
---------------------------------------------------------------------
Optimal treshold for CV 1 is 7.872691034391308
              precision    recall  f1-score   support

           0       0.97      0.95      0.96       293
           1       0.75      0.83      0.79        52

    accuracy                           0.93       345
   macro avg       0.86      0.89      0.87       345
weighted avg       0.94      0.93      0.93       345

 MCC: 0.750616576846467, Accuracy: 0.9333333333333333, F1: 0.7889908256880734
--------------------------

In [ ]:
optimal_treshold = np.mean(optimal_tresholds)
optimal_treshold

8.205864702875445

# 3. Method testing

In [ ]:
test_fold_true_values_df = test_df.loc[test_df['Class'] == 'SP']
pspm = build_pspm(test_fold_true_values_df)
n = len(test_fold_true_values_df) + 20
pswm = build_pswm(pspm, n=n)

In [ ]:
# prediction
y_test_true = test_df['Class'].apply(lambda x: 1 if x == 'SP' else 0).values
y_test_pred = test_df['Sequence (first 50 N-terminal residues)'].apply(predict_score, pswm=pswm).values
y_test_pred = [int(y >= optimal_treshold) for y in y_test_pred]

In [ ]:
print('Optimal treshold for benchmark dataset is {}'.format(optimal_treshold))
calculate_all_metrics(y_test_true, y_test_pred)
print('---------------------------------------------------------------------')


Optimal treshold for benchmark dataset is 8.205864702875445
              precision    recall  f1-score   support

           0       0.99      0.97      0.98      7247
           1       0.48      0.82      0.61       209

    accuracy                           0.97      7456
   macro avg       0.74      0.90      0.80      7456
weighted avg       0.98      0.97      0.97      7456

 MCC: 0.6163883591639759, Accuracy: 0.9704935622317596, F1: 0.6085409252669038
---------------------------------------------------------------------


In [ ]:
from sklearn import datasets
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
precision = precision_score(y_test_true, y_test_pred)
recall = recall_score(y_test_true, y_test_pred)
mcc=matthews_corrcoef(y_test_true, y_test_pred)
acc=accuracy_score(y_test_true, y_test_pred)
print('Precision:',precision)
print('Recall:' ,recall)
print('MCC:',mcc)
print('ACC:',acc)

Precision: 0.48441926345609065
Recall: 0.8181818181818182
MCC: 0.6163883591639759
ACC: 0.9704935622317596


In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix = confusion_matrix(y_test_true, y_test_pred)
print(conf_matrix)

[[7065  182]
 [  38  171]]
